# NA 568 Mobile Robotics: Methods \& Algorithms Winter 2021 -- Homework 2 -- Kalman Filtering

University of Michigan\
January 25, 2021

## Task 4 First-Order Covariance Propagation (25 points)

In most SLAM approaches, the noise characteristics of individual observations are linearized, resulting in a Gaussian covariance matrix. In this task, we explore the effects of the error introduced via linearization.

Consider a robot sensor that observes range and bearing to nearby landmarks. In this case, the range error is relatively small, but the bearing error is large. We are interested in determining the $(x, y)$ position of the beacon based on observations obtained by the robot at the origin $(0, 0)$. For this problem, the robot does not move.

Suppose you obtain a (range, bearing) observation with mean ($10.0$ m, $0$ rad) whose range standard deviation is $0.5$ m, and whose bearing standard deviation is $0.25$ rad. The range and bearing measurements are Gaussian and independent.

### A. (4 pts) 
Generate a point cloud representing 10,000 samples from the distribution over the position of the beacon as measured in\
  i) the sensor frame, i.e. $(r, \theta)$ space and\
  ii) the Cartesian $(x, y)$ coordinate frame.

In other words, generate observations of (range, bearing) and project these points into $(x,y)$. (Hint: use the *numpy.random.randn()* or *numpy.random.multivariate_normal()* function, and recall that you can sample from a univariate Gaussian with mean *mu* and standard deviation *sigma* with *mu + sigma* \* *randn*.)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# parameter setting
N = 10000
mu_sensor = [10, 0]
sigma_sensor = [0.5, 0.25]

# generate point clouds
r, theta = np.zeros(N), np.zeros(N)
x, y = np.zeros(N), np.zeros(N)
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
# i) Sensor (r, theta) frame 
for i in range(N):
    r[i] = np.random.normal(mu_sensor[0],sigma_sensor[0])
    theta[i] = np.random.normal(mu_sensor[1],sigma_sensor[1])
    x[i] = np.cos(theta[i])
    y[i] = np.sin(theta[i])


# ii) Cartesian (x,y) coordinate frame


#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################
# i) Observation in the sensor frame
plt.plot(r, theta, '.', markersize=2)
plt.axis('equal')
plt.grid(True)
plt.title('Sensor Frame Point Cloud')
plt.xlabel('Range (m)')
plt.ylabel('Bearing (rad)')
plt.show()

# ii) Observation in the Cartesian frame
plt.plot(x, y, '.', markersize=2)
plt.axis('equal')
plt.grid(True)
plt.title('Cartesian Frame Point Cloud')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.show()

### B. (4 pts)
What is the (linearized) covariance of the beacon position in $(x, y)$ coordinates? In other words,   write the covariance of an observation in $(x,y)$ coordinates in terms of the covariance of the   observation in (range, bearing) coordinates. The transformation is non-linear, so you will need to compute a first-order approximation (Taylor expansion) of the transformation function. Make the   appropriate Jacobians easy to read in your source code, using comments if necessary.

In [ ]:
Jacobian = np.zeros((2,2))
cov_cartesian = np.zeros((2,2))
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
# Implement the Jacobians
Jacobian = np.array[[1, 0],[0, 10]]
cov_cartesian = Jacobian@np.array[[0.25, 0],[0, 0.0625]]@Jacobian


# Implement the linearized covariance in cartesian corridinates


#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################
print('Jacobian:\n', Jacobian)
print('\nSigma_cartesian:\n', cov_cartesian)

### C. (4 pts)
Draw in red the 1-sigma, 2-sigma, and 3-sigma contours of the analytical (linearized) covariance ellipses, super-imposed over the point clouds generated in parts A.i and A.ii.  Now overlay in blue   the actual covariance ellipses computed using sample-based expressions for the first and second moments.  Do they agree?  Why or why not?

You may use the function *draw\_ellipse()* provided bellow.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def calculateEllipseXY(mu, Sigma, k2, N=20):
    """
    input:
    mu     is the [2x1] [x;y] mean vector
    Sigma  is the [2x2] covariance matrix
    k2     is the Chi-squared 2 DOF variable
    N      is the number of points to use (optional, default 20)
    
    output:
    x and y coordinates to draw the ellipse
    """
    # set up angles for drawing the ellipse
    angles = np.linspace(0, 2*np.pi, num=N)
    _circle = np.array([np.cos(angles), np.sin(angles)])

    # make sure it is a numpy array
    mu = np.array(mu)
    Sigma = np.array(Sigma)
        
    # cholesky decomposition
    L = np.linalg.cholesky(Sigma) # Cholesky factor of covariance
    
    # apply the transformation and scale of the covariance
    ellipse = np.sqrt(k2) * L @ _circle

    # shift origin to the mean
    x = mu[0] + ellipse[0, :].T
    y = mu[1] + ellipse[1, :].T

    return x, y

def draw_ellipse(mu, Sigma, k2, colorin='red'):
    """   
    input:
    mu       is the [2x1] [x;y] mean vector
    Sigma    is the [2x2] covariance matrix
    k2       is the Chi-squared 2 DOF variable
    Npoints  number of points to draw the ellipse (default 20)
    colorin  color for plotting ellipses, red for analytical contours, blue for sample contours
    
    --- h = draw_ellipse(mu, Sigma, k2)
    Draws an ellipse centered at mu with covariance Sigma and confidence region k2, i.e.,
    K2 = 1; # 1-sigma
    K2 = 4; # 2-sigma
    K2 = 9; # 3-sigma
    K2 = chi2inv(.50, 2); # 50% probability contour
    """
    Npoints = 20
    
    x, y = calculateEllipseXY(mu, Sigma, k2, Npoints)
    
    if k2 == 9:
        if colorin == 'red':
            plt.plot(x, y, linewidth=1.25, color=colorin, label='analytical contours')
        elif colorin == 'blue':
            plt.plot(x, y, linewidth=1.25, color=colorin, label='sample contours')
    else:
        plt.plot(x, y, linewidth=1.25, color=colorin)

In [ ]:
# Sensor frame
plt.plot(r, theta, '.', markersize=2, label='data')
plt.axis('equal')
plt.grid(True)
plt.title('Sensor Frame Point Cloud')
plt.xlabel('Range (m)')
plt.ylabel('Bearing (rad)')
plt.legend()

# compute the mean and covariance of samples in the sensor frame 
# mu_sensor = np.zeros((2,1))
# cov_sensor = np.zeros((2,2))
# mu_sensor_sample = np.zeros((2,1))
# cov_sensor_sample = np.zeros((2,2))
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
mu_sensor = np.matrix([[10],[0]])
cov_sensor = np.matrix([[0.25, 0],[0, 0.0625]])

mu_r = sum(r)/r.size()
mu_theta = sum(theta)/theta.size()
mu_sensor_sample = np.matrix([[mu_r],[mu_theta]])

cov_r = 0
for i in r:
    cov_r += (i - mu_r)**2/(r.size()-1)
cov_theta = 0
for j in theta:
    cov_theta += (j - mu_theta)**2/(theta.size()-1)
cov_sensor_sample = np.matrix([[cov_r, 0],[0, cov_theta]])


#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

# plot the ellipses
for i in range(3):
    # analytical contous in red
    draw_ellipse(mu_sensor, cov_sensor, (i+1)**2, 'red')
    # sample contous in blue
    draw_ellipse(mu_sensor_sample, cov_sensor_sample, (i+1)**2, 'blue')

plt.legend()
plt.show()

# Cartesian frame
plt.plot(x, y, '.', markersize=2, label='data')
plt.axis('equal')
plt.grid(True)
plt.title('Cartesian Frame Point Cloud')
plt.xlabel('x (m)')
plt.ylabel('y (m)')

# compute the mean and covariance of samples in the Cartesian frame
# mu_cartesian = np.zeros((2,))
# cov_cartesian = np.zeros((2,2))
# mu_cartesian_sample = np.zeros((2,))
# cov_cartesian_sample = np.zeros((2,2))
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
mu_cartesian = np.matrix([[10],[0]])
mu_cartesian_sample = np.matrix([[0.25, 0],[0, 0.0625]])

mu_x = sum(x)/x.size()
mu_y = sum(y)/y.size()
mu_cartesian_sample = np.matrix([[mu_x],[mu_y]])


cov_x = 0
for i in x:
    cov_x += (i - mu_x)**2/(x.size()-1)
cov_y = 0
for j in y:
    cov_y += (j - mu_y)**2/(y.size()-1)
cov_cartesian_sample = np.matrix([[cov_x, 0],[0, cov_y]])
#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

# plot the ellipses
for i in range(3):
    # analytical contous in red
    draw_ellipse(mu_cartesian, cov_cartesian, (i+1)**2, 'red')    
    # sample contous in blue
    draw_ellipse(mu_cartesian_sample, cov_cartesian_sample, (i+1)**2, 'blue')

plt.legend()
plt.show()

### D. (4 pts)
From a purely theoretical perspective, assuming that the underlying process is truly Gaussian, we expect 39.35% of all samples to lie within the 1-sigma contour, 86.47\% of samples to lie within the 2-sigma contour, and 98.89% to lie within the 3-sigma contour. (These frequencies were computed using the cumulative chi-square distribution for two degrees-of-freedom)

Modify your software to count the samples falling within each (analytical) ellipse for parts A.i   and A.ii. The error of a particular sample $x$, measured in "units" of sigma, is known as the Mahalanobis distance, and can be computed as $\sqrt{(x-\mu)^\top\Sigma^{-1}(x-\mu)}$.

In [ ]:
# counter for samples lie within the contour
count_sensor = np.zeros((3,1)) # count results of samples in the sensor frame
count_cartesian = np.zeros((3,1)) # count results of samples in the Cartesian frame

# Compute the Mahalabobis distance of samples, and count how many samples lie in the contour
#############################################################################
#                    TODO: Implement your code here                         #
#############################################################################
mu_sensor = np.matrix([[10],[0]])
cov_sensor = np.matrix([[0.25, 0],[0, 0.0625]])
r1,r2,r3 = 0
for i in range(N):
    x = np.matrix([[r[i]],[theta[i]]])
    error_sensor = np.sqrt((x-mu_sensor).T*np.linalg.inv(cov_sensor)*(x-mu_sensor))
    if error_sensor < 1:
        r1 += 1
    if error_sensor < 2:
        r2 += 1
    if error_sensor < 3:
        r3 += 1

count_sensor[0] = r1
count_sensor[1] = r2
count_sensor[2] = r3

mu_cartesian = np.matrix([[10],[0]])
cov_cartesian = np.matrix([[0.25, 0],[0, 6.25]])
k1,k2,k3 = 0
for i in range(N):
    m = np.matrix([[x[i]],[y[i]]])
    error_sensor = np.sqrt((m-mu_cartesian).T*np.linalg.inv(cov_sensor)*(m-mu_cartesian))
    if error_sensor < 1:
        k1 += 1
    if error_sensor < 2:
        k2 += 1
    if error_sensor < 3:
        k3 += 1

count_cartesian[0] = k1
count_cartesian[1] = k2
count_cartesian[2] = k3


    








#############################################################################
#                            END OF YOUR CODE                               #
#############################################################################

# print out the result
Gaussian = [0.39, 0.86, 0.99]
print("         Sensor frame  Cartesian frame  Gaussian\n")
for k in range(3):
    print('%d-sigma%10.2f%16.2f%13.2f\n' % (k+1, count_sensor[k]/N, count_cartesian[k]/N, Gaussian[k]))
    


### F. (5 pts)
Suppose now that the (range, bearing) measurements are *not independent* but instead jointly correlated under the following three scenarios: \
a) $\rho_{{r\theta}}=0.1$, \
b) $\rho_{r\theta}=0.5$, and \
c) $\rho_{r\theta}=0.9$. 

Repeat parts A and C.

In [ ]:
for rho in [0.1, 0.5, 0.9]: # correlation coefficient, eg. 0.1, 0.5, 0.9
    print('rho =', rho)
    
    # Part A. generate point clouds
    # r, theta = np.zeros(N), np.zeros(N)
    # x, y = np.zeros(N), np.zeros(N)
    #############################################################################
    #                    TODO: Implement your code here                         #
    #############################################################################
    mu_sensor = np.matrix([[10],[0]]) 
    cov_sensor = np.matrix([[0.25, rho],[rho, 0.0625]])
    data_sensor = np.random.multivariate_normal(mu_sensor,cov_sensor,N)
    r = 

    
    #############################################################################
    #                            END OF YOUR CODE                               #
    #############################################################################
    
    # Part C. Draw ellipse
    # Sensor frame
    plt.plot(r, theta, '.', markersize=2, label='data')
    plt.axis('equal')
    plt.grid(True)
    plt.title(r'Sensor Frame Point Cloud with $\rho$ = %.1f' % rho)
    plt.xlabel('Range (m)')
    plt.ylabel('Bearing (rad)')
    plt.legend()

    # compute the mean and covariance of samples in the sensor frame 
    mu_sensor = np.zeros((2,))
    cov_sensor = np.zeros((2,2))
    mu_sensor_sample = np.zeros((2,))
    cov_sensor_sample = np.zeros((2,2))
    #############################################################################
    #                    TODO: Implement your code here                         #
    #############################################################################
    
    
    #############################################################################
    #                            END OF YOUR CODE                               #
    #############################################################################

    # plot the ellipses
    for i in range(3):
        # analytical contous in red
        draw_ellipse(mu_sensor, cov_sensor, (i+1)**2, 'red')
        # sample contous in blue
        draw_ellipse(mu_sensor_sample, cov_sensor_sample, (i+1)**2, 'blue')

    plt.legend()
    plt.show()

    # Cartesian frame
    plt.plot(x, y, '.', markersize=2, label='data')
    plt.axis('equal')
    plt.grid(True)
    plt.title(r'Cartesian Frame Point Cloud with $\rho$ = %.1f' % rho)
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')

    # compute the mean and covariance of samples in the Cartesian frame
    mu_cartesian = np.zeros((2,))
    cov_cartesian = np.zeros((2,2))
    mu_cartesian_sample = np.zeros((2,))
    cov_cartesian_sample = np.zeros((2,2))
    #############################################################################
    #                    TODO: Implement your code here                         #
    #############################################################################
    
    
    #############################################################################
    #                            END OF YOUR CODE                               #
    #############################################################################

    # plot the ellipses
    for i in range(3):
        # analytical contous in red
        draw_ellipse(mu_cartesian, cov_cartesian, (i+1)**2, 'red')    
        # sample contous in blue
        draw_ellipse(mu_cartesian_sample, cov_cartesian_sample, (i+1)**2, 'blue')

    plt.legend()
    plt.show()